In [4]:
import re
import fitz  # PyMuPDF
pdf_path = r"C:\Users\Dell\OneDrive\Documents\Desktop\Unani project\pharmacopia Part 2 volume 1.pdf"
doc = fitz.open(pdf_path)
entries = re.split(r"\n[A-Za-z-]+\(NFUM.*\)\n", pdf_path)

start_page = 26
text = ""
for i in range(start_page, len(doc)):
    text += doc[i].get_text("text")+"\n"

print("preview of starting page text:\n")
print(text[:500])

preview of starting page text:

3
SAIYYALAT
Definition:
1. Saiyyalat are those medicinal preparations where the drugs of plants, animal and mineral
origin are used in liquid forms which are broadly classified as (i) Araq (Distillate) (ii) Qutur
(Eye Drops) and (iii) Sharbat (Syrup).
Various processes are adopted depending on the type and nature of the liquid preparations.
2. (i)
Araqiyat are those liquid preparations which are made by using the drugs of plant, animal
and mineral origin by the process of distillation involving 


In [5]:
import fitz  # PyMuPDF
import re

pdf_path = r"C:\Users\Dell\OneDrive\Documents\Desktop\Unani project\pharmacopia Part 2 volume 1.pdf"
doc = fitz.open(pdf_path)

start_page = 28
end_page = 153  # inclusive

text = ""
for i in range(start_page, end_page + 1):  # +1 to include page 129
    text += doc[i].get_text("text") + "\n"

# === Save to TXT file ===
output_path = "5Unani_extract.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(text)

print(f"✅ Text extracted from page {start_page} to {end_page} and saved as {output_path}")

# === Preview of first 500 characters ===

print(text[:500])


✅ Text extracted from page 28 to 153 and saved as 5Unani_extract.txt
5
ARAQ-E-AJEEB
(NFUM-I, 9.2)
Definition:
Araq-e-Ajeeb is a liquid preparation obtained by mixing Kafoor, Jauhar-e-Pudina and Jauhar-e-
Ajwayin at room temperature in the quantity given below.
Formulation composition:
1.
Kafoor
Camphor, UPI
Crystals
40 g
2.
Jauhar-e-Pudina
Menthol, API
Crystals
40 g
3.
Jauhar-e-Ajwayin
Thymol, API
Crystals
20 g
Method of preparation:
Take all the three ingredients of pharmacopoeial quality. Crush them separately to make their  coarse
powder, then mix as per compo


In [2]:
import re

with open("5Unani_extract.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Split whenever a new medicine starts
entries = re.split(r"\n[A-Za-z\s-]+\n\(NFUM.*\)", text)

print(f"Found {len(entries)} medicines")
print(entries[40])  # preview first medicine

Found 47 medicines

Definition:
Raughan-e-Malkangni is an oil obtained by cold expelling process from Malkangni seeds.
Formulation composition:
1. Malkangni
Celastrus paniculatus Willd., UPI
Seed
Q.S.
Method of preparation:
Take the ingredient of pharmacopoeial quality.
Clean and dry under shade. Put the seeds of Malkangni into expeller for getting the oil content by
cold expelling process. Filter the oil through a muslin cloth and then store in dry containers protected
from light and moisture.
Description:
The drug Raughan-e-Malkangni is viscous liquid of dark brown colour with pungent smell and
unpleasant taste.
Identification:
Thin Layer Chromatography:
TLC of the drug (as such) on precoated aluminium plate of silica gel 60 F-254 using toluene: ethyl
acetate (9:1) shows five spots at Rf 0.23 (Pinkish purple), 0.32 (Purple), 0.51 (Light brown), 0.54
(Light brown) and 0.60 (Yellowish brown) on spraying with 5% ethanolic sulphuric acid and heating
the plate for about ten minutes at 105

In [7]:
import re
import pandas as pd

with open("5Unani_extract.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Split into medicines
medicine_pattern = re.compile(
    r"([A-Z\s-]+)\r?\n(\(NFUM[^\n]+\))\s*(.*?)(?=(?:[A-Z\s-]+\r?\n\(NFUM)|\Z)", 
    re.S
)
medicines = medicine_pattern.findall(text)

print(f"Found {len(medicines)} medicines")

# Canonical headers
headers = [
    "Definition", "Formulation composition", "Method of preparation", "Description",
    "Identification", "Physico-chemical parameters", "Microbial load", "Aflatoxins",
    "Pesticidal residue", "Heavy metals", "Storage", "Therapeutic uses",
    "Actions", "Dose", "Mode of administration"
]

# Map variations → canonical headers
header_map = {
    "definition": "Definition",
    "formulation composition": "Formulation composition",
    "composition": "Formulation composition",
    "composition of formulation": "Formulation composition",
    "method of preparation": "Method of preparation",
    "preparation": "Method of preparation",
    "description": "Description",
    "identification": "Identification",
    "tests of identification": "Identification",
    "physico chemical parameters": "Physico-chemical parameters",
    "physico-chemical parameters": "Physico-chemical parameters",
    "physicochemical parameters": "Physico-chemical parameters",
    "parameters": "Physico-chemical parameters",
    "microbial load": "Microbial load",
    "aflatoxins": "Aflatoxins",
    "pesticidal residue": "Pesticidal residue",
    "heavy metals": "Heavy metals",
    "storage": "Storage",
    "therapeutic uses": "Therapeutic uses",
    "uses": "Therapeutic uses",
    "actions": "Actions",
    "dose": "Dose",
    "mode of administration": "Mode of administration",
    "administration": "Mode of administration"
}

# Extract sections
def extract_sections(entry):
    sections = {}

    # Build regex dynamically
    header_regex = "|".join(sorted(header_map.keys(), key=len, reverse=True))

    # Find all headers and capture until the next
    found = re.findall(
        rf"({header_regex})\s*:?\s*(.*?)(?=(?:\r?\n(?:{header_regex})\s*:?)|\Z)",
        entry, re.S | re.I
    )

    for header, content in found:
        norm = header.strip().lower().replace("-", " ").replace("  ", " ")
        if norm in header_map:
            sections[header_map[norm]] = content.strip()

    return sections

rows = []
for name, nfum, entry in medicines:
    sec_data = extract_sections(entry)
    row = {"Name": name.strip(), "NFUM code": nfum.strip()}
    for h in headers:
        row[h] = sec_data.get(h, "")
    rows.append(row)

# Save
df = pd.DataFrame(rows)
df.to_excel("Y-Unani_medicines.xlsx", index=False)

print(f"✅ Saved {len(df)} medicines to AF-Unani_medicines.xlsx")


Found 50 medicines
✅ Saved 50 medicines to AF-Unani_medicines.xlsx


In [8]:
import pandas as pd

# Load your Excel file
file_path = "Y-Unani_medicines.xlsx"
df = pd.read_excel(file_path)

# Find empty cells
empty_cells = df.isnull()

# Print coordinates of empty cells
for row in range(len(df)):
    for col in df.columns:
        if empty_cells.loc[row, col]:
            print(f"Empty cell found at Row {row+2}, Column '{col}'")


Empty cell found at Row 5, Column 'Identification'
Empty cell found at Row 6, Column 'Physico-chemical parameters'
Empty cell found at Row 7, Column 'Physico-chemical parameters'
Empty cell found at Row 9, Column 'Physico-chemical parameters'
Empty cell found at Row 9, Column 'Actions'
Empty cell found at Row 10, Column 'Actions'
Empty cell found at Row 14, Column 'Actions'
Empty cell found at Row 15, Column 'Physico-chemical parameters'
Empty cell found at Row 15, Column 'Actions'
Empty cell found at Row 16, Column 'Physico-chemical parameters'
Empty cell found at Row 16, Column 'Actions'
Empty cell found at Row 17, Column 'Therapeutic uses'
Empty cell found at Row 18, Column 'Description'
Empty cell found at Row 18, Column 'Actions'
Empty cell found at Row 19, Column 'Description'
Empty cell found at Row 21, Column 'Actions'
Empty cell found at Row 24, Column 'Identification'
Empty cell found at Row 28, Column 'Actions'
Empty cell found at Row 33, Column 'Therapeutic uses'
Empty cell